# Peak shaving und Optimierung

Das Stromnetz ist dimensioneiert, um einen zuverlässigen Dienst zu erfüllen und dennoch in Kosten beherschbar zu bleiben. Das bedeutet oft, dass die Komponenten so dimensioniert sind, dass sie den höchsten *zu erwartenden* Bedarf decken, auch wenn dieser nur selten auftritt. Hohe Lastspitzen sind unerwünscht, da sie zu einer überdimensionierten und teuren Infrastruktur führen, um Netzunterbrechungen zu vermeiden.

Um Großverbraucher dazu zu bewegen, ihre Leistungsspitzen zu reduzieren, werden auf ihrer Stromrechnung neben den regulären Energieverbrauchskosten auch Gebühren für die während des Abrechnungszeitraums (z.B. ein Jahr, z.T. auch ein Monat) gemessene Höchstleistung ausgewiesen.
Die Verbraucher könnten diesen "Leistungspries" senken, indem sie  z.B. ihr Verbrauchsverhalten anpassen und flexible Lasten in verbrauchsarmen Zeiten verschieben.
Eine Alternative, ist die Nutzung eines Batteriespeichersystems, um die zeitliche Abfolge der Lastbedarfs zu verschieben und Spitzen abzufangen.
Im Ergebnis kann ein Lastprofil "abgeflacht" werden, während der Verbrauch gleich bleibt (bzw. sich wegend des Eigenverbrauchs des Speichers leicht erhöt).

In diesem Jupyter Notebook werden wir:
* Das Peak-Shaving mittels Batteriespeicherals als Möglichkeit zur Reduzierung der Stromspitzenkosten genauer analysieren.
* Lineare Optimierungsmodelle einsetzen, um eine *Steuerungsstrategie* für das Batteriesystem zur Spitzenlastreduzierung zu entwerfen.
* Das o.g. Modell erweitern um eine Möglichkeit auch die *Dimensionierung* unseres Speichersystems zu optimieren.

In [ ]:
import pandas as pd
import pyomo.environ as opt
import plotly.graph_objects as go

In [ ]:
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Use case - dataset and base-scenario definition
For our example we take an historic load profile from HS Kempten Campus. For simplicity, we will consider a single billing period of 1 year.

In [ ]:
# use 15 min time step:
# optimization time:
#   task 1:     25s
#   task 2:     3m11s * 2
# profile =df_15min

# use 1h time step:
# optimization time:
#   task 1:     5s
#   task 2:     28s * 2 
dt = 0.25 # h

if dt == 0.25:    
    # load profile: version: 1h / 15 min time step version
    profile = pd.read_csv("../data/HSK/HSK_22_15min.csv", index_col=['time_date'], parse_dates=True).drop(columns=['Unnamed: 0','time']).rename(columns={'pUser': 'load'})

if dt == 1:
    # load profile: version: 1h / 15 min time step version
    profile = pd.read_csv("../data/HSK/HSK_22_1h.csv", index_col=0, parse_dates=True).rename(columns={'pUser': 'load'})

profile.plot(template=template, labels={"value": "Power [kW]"})

<div class="alert alert-block alert-info">
<b>Task IV </b> Calculate energy consumption, peak power and the resulting electricity bill.
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Hint! Pay attention when converting power to energy. </li>
</div>

In [ ]:
# cost
electricity_cost =   0.12 # €/kWh
peak_power_cost  = 120.00 # €/kW

In [ ]:
# task IV: Calculate energy consumption, peak power and the resulting electricity bill.
def baseline_analysis(profile, dt, electricity_cost, peak_power_cost):
    demand = sum(profile["pRes"]*dt)
    peak = max(profile["pRes"])
    cost =   peak*peak_power_cost+ demand*electricity_cost                
    print(f"Consumption: {demand:10.2f} kWh")
    print(f"Peak power:  {peak:10.2f} kW")
    print(f"Total costs: {cost:10.2f} €")

In [ ]:
print("--- without storage ---")
baseline_analysis(profile, dt, electricity_cost,peak_power_cost)    

## Optimization model - Operation

An energy storage system can reduce costs by performing peak shaving. But how exactly should it operate? 

We can define a target maximum power peak and charge or discharge the storage depending on if the load is currently above or below this threshold.
Defining a peak target could be nonetheless difficult, if the threshold is too high the storage system may not be used at its full potential, if it is too low, the storage system would not be able to fulfil it.

We can instead use mathematical optimization techniques to find the optimal storage charge/discharge schedule to best fulfill the task. The [*pyomo* library](https://www.pyomo.org/) allows us to easily formulate *linear programming* models with python.

<div class="alert alert-block alert-info">
<b>Task V</b> Formulate and optimize the model
<ol>
    <li> Formulate the optimization problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the time series results togther in a dataframe with the load profile. Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs. Evaluate the improvements against the scenario without storage system. </li>
    
</ol>
</div>

<div class="alert alert-block alert-info">
TASK V.1: Formulate the optimization problem in a markdown cell using \( \LaTeX \)

<b>Hint!</b> Read the <a href="https://pyomo.readthedocs.io/en/stable/">pyomo documentation</a>. </li>
</div>

$$
\begin{aligned}
&\min_{a, b, c} \quad f(b, c) = 2 * \sum_{T}{b_t} + c^2 \\
s.t.  &  \\
b_t &\leq a_t, \quad &\forall t \in T \\
b_t &\geq c, \quad &\forall t \in T \\
c &= d,  \\
\end{aligned}
$$

In [ ]:
# Storage parameters - Definition as "Dictionary"
storage_params = {
    "capacity": 200.0,  # kWh
    "power": 200.0,      # kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.1,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

In [ ]:
storage_params["effc"]

<div class="alert alert-block alert-info">

### Task V.2:  Create a function <code>build_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>.


In [ ]:
# Task V.2: Build the model based on the given storage parameters and optimize it with the GLPK solver.
def build_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # Optimization model
    model = opt.ConcreteModel()
    ## =====  Parameters  ===== ##
    n = len(load_profile)   # number of timesteps
    model.time = opt.RangeSet(0, n-1) # T
    max_power = storage_params["power"]
    ...

    ## =====  Variables  ===== ##
    # BESS: power charge/discharge, energy content
    model.power_charge    = opt.Var(model.time, bounds=(0.0, max_power))
    ...

    # Grid: power, peak-power limit (for peak-shaving)

    ## =====  Objective  ===== ##
    ...
    model.obj = opt.Objective(sense=opt.minimize,.......)

    ## ===== Constraints ===== ##
   # power balance
    def power_balance_rule(m, t):
        .....
    
    return model

In [ ]:
# task V.3: build model
model = build_model(
    ...
)

In [ ]:
model.pprint()

In [ ]:
solver = opt.SolverFactory('glpk')  # glpk is an open source LP solver

In [ ]:
status = solver.solve(model, tee=True)

In [ ]:
# taskV.4: recover results
def recover_results(model, profile):
    df = profile.copy() # Solutions dataframe
    df["BESS_power"] =
    df["BESS_energy"] =
    df["grid_power"] =
    return df

In [ ]:
# task V.4: plot results 

def plot_storage_power(profile, model):
    ...
    return fig

In [ ]:
# PLOT: Load profile and grid power in the same plot.

In [ ]:
# PLOT:  Storage power. 


In [ ]:
# PLOT:  Storage energy content or SOE (i.e. State of Energy).

In [ ]:
# task V.5: analyze results
def analyze_results(...):
    
    ...

    print(f"Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total costs: {opt_cost:10.2f} €   ({delta_cost:+7.2%})")

<div class="alert alert-block alert-success">
Weitere Tasks in Diskussion erst nach dem 27.5.24


## System dimension

We have succesfully optimized our storage operation to minimizes costs. This was done with a predefined system. But what if we find ourselfs on the planning stage, contemplating the idea of aquiring a storage system to perform peak shaving? The storage system comes with respective investment costs, so me might want to 
re-formulate the optimization problem with a further degree of freedom to find the best system size that minimizes the total costs.

Two storage system are considered, their ANNUAL specific costs and efficiency are described in the following table:

|                                   | Storage System 1  | Sotrage System 2  |
|:---------------------------------:|:-----------------:|:-----------------:|
| Specific capacity costs           | 50.00 €/kWh       | 20.00 €/kWh       |
| Specific power costs              | 20.00 €/kW        | 50.00 €/kW        |
| Efficiency charge ; discharge     | 95% ; 95%         | 90%  ; 90%        |

<div class="alert alert-block alert-info">
<b>Task!</b> Formulate and optimize the dimensioning model
<ol>
    <li> Formulate the optimal sizing problem in a markdown cell using \( \LaTeX \). </li>
    <li> Create a function <code>build_dimension_model</code> that takes storage system parameters <code>storage_params</code>, load profile <code>load_profile</code>, electricity costs <code>electricity_cost</code>, peak costs <code>peak_power_cost</code> and returns a pyomo optimization model <code>model</code>. </li>
    <li> Build the model based on the given storage parameters and optimize it with the GLPK solver. </li>
    <li> Recover the optimization results. Combine the load profile and the time series results of both storage systems in a dataframe. Make the following plots:
    <ol>
        <li> Load profile and grid power in the same plot. </li>
        <li> Storage power. </li>
        <li> Storage energy content or SOC. </li>
    </ol>
    </li>
    <li> Calculate the new total consumption, peak-power and total costs for both systems. Which system would you choose? Give your arguments in a markdown cell. </li>
    
</ol>
</div>

TASK 1: LATEX


...
...
...


**Variables**: Storage system installed capacity $E^{N}$, storage system installed power $p^{N}$, energy charge state at $t$ timestep $E^{bess}$, charge power at $t$ timestep $p^{ch}_t$, discharge power at $t$ timestep $p^{ch}_t$, grid power supply at $t$ timestep $p^{grid,buy}_t$, peak power $p^{peak}$.

**Parameters**: Energy capacity specific investment costs $k^{inv,E}$, power specific investment costs $k^{inv,E}$, electricity cost $k^{electricity}$, peak power cost $p^{peak}$, lower SOC limit $SOC^{min}$, upper SOC limit $SOC^{min}$, start SOC value $SOC^{init}$, charge efficiency $\eta^{ch}$, discharge efficiency $\eta^{dch}$, power demand at $t$ timestep $p^{load}_t$.

In [ ]:
# Storage parameters
storage1_params = {
    "capacity_cost": 50.00,  # €/kWh
    "power_cost": 20.00,     # €/kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.0,

    "effc": 0.95,     # charge efficiency
    "effd": 0.95,     # discharge efficiency
}

storage2_params = {
    "capacity_cost": 20.00,  # €/kWh
    "power_cost": 50.00,     # €/kW

    "soc_bounds": (0.1, 0.9),
    "soc_start": 0.0,

    "effc": 0.9,     # charge efficiency
    "effd": 0.9,     # discharge efficiency
}

In [ ]:
def build_dimension_model(storage_params, load_profile, electricity_cost, peak_power_cost):
    # task
    ...

    return model

In [ ]:
model_dim1 = build_dimension_model(
    ...
)

In [ ]:
#solve model 1
status = solver.solve(model_dim1, tee=True)

In [ ]:
model_dim2 = build_dimension_model(
    ...
)

In [ ]:
#solve model 2
status = solver.solve(model_dim2, tee=True)

In [ ]:
# task: recover results
...

In [ ]:
# task: analyze results
def analyze_dimension_results(...):
    # system dimension
    ...
    # operation
    ...
    
    
    print(f"Installed capacity: {capacity:7.2f} kWh")
    print(f"Installed power:    {power:7.2f} kW")
    print(f"System costs:       {system_cost:7.2f} €\n")
    
    print(f"Consumption: {opt_demand:10.2f} kWh ({delta_demand:+7.2%})")
    print(f"Peak power:  {opt_peak:10.2f} kW  ({delta_peak:+7.2%})")
    print(f"Total costs: {opt_cost:10.2f} €   ({delta_cost:+7.2%})")



In [ ]:
# task: analyze resuls
print("--- storage 1 ---")
analyze_dimension_results(res_dim1, model_dim1, storage1_params, electricity_cost, peak_power_cost)
print()
print("--- storage 2 ---")
analyze_dimension_results(res_dim2, model_dim2, storage2_params, electricity_cost, peak_power_cost)

In [ ]:
# task: plot results